In [1]:
# @title Update/Upgrade the system and install libs
'''
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install -y swig build-essential python-dev python3-dev > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!apt-get install xvfb > /dev/null 2>&1
'''
# !pip install moviepy

'\n!apt-get update > /dev/null 2>&1\n!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1\n!apt-get install -y swig build-essential python-dev python3-dev > /dev/null 2>&1\n!apt-get install x11-utils > /dev/null 2>&1\n!apt-get install xvfb > /dev/null 2>&1\n'

In [2]:
# @title Install dependencies
'''
!pip install rarfile
!pip install stable-baselines3[extra]
!pip install ale-py
!pip install gym[box2d]
!pip install pyvirtualdisplay
!pip install pyglet
!pip install pygame
!pip install minigrid
!pip install -q swig
!pip install -q gymnasium[box2d]
!pip install 'minigrid<=2.1.1'
!pip3 install box2d-py
'''

"\n!pip install rarfile\n!pip install stable-baselines3[extra]\n!pip install ale-py\n!pip install gym[box2d]\n!pip install pyvirtualdisplay\n!pip install pyglet\n!pip install pygame\n!pip install minigrid\n!pip install -q swig\n!pip install -q gymnasium[box2d]\n!pip install 'minigrid<=2.1.1'\n!pip3 install box2d-py\n"

In [3]:
# Imports
import io
import os
import glob
import torch
import base64

import numpy as np
import matplotlib.pyplot as plt

import sys
import gymnasium
sys.modules["gym"] = gymnasium

import stable_baselines3
from stable_baselines3 import DQN
from stable_baselines3.common.results_plotter import ts2xy, load_results
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_atari_env

import gymnasium as gym
from gym import spaces
from gym.envs.box2d.lunar_lander import *
from gym.wrappers.monitoring.video_recorder import VideoRecorder

In [4]:
# @title Play Video function
from IPython.display import HTML
from base64 import b64encode
from pyvirtualdisplay import Display

# create the directory to store the video(s)
os.makedirs("./video", exist_ok=True)

display = Display(visible=False, size=(1400, 900))
_ = display.start()

"""
Utility functions to enable video recording of gym environment
and displaying it.
To enable video, just do "env = wrap_env(env)"""
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

In [5]:
nn_layers = [64, 64]  # This is the configuration of your neural network. Currently, we have two layers, each consisting of 64 neurons.
                      # If you want three layers with 64 neurons each, set the value to [64,64,64] and so on.

learning_rate = 0.001  # This is the step-size with which the gradient descent is carried out.
                       # Tip: Use smaller step-sizes for larger networks.

In [6]:
log_dir = "/tmp/gym/"
os.makedirs(log_dir, exist_ok=True)

# Create environment
env_name = 'LunarLander-v2'
env = gym.make(env_name)
# You can also load other environments like cartpole, MountainCar, Acrobot.
# Refer to https://gym.openai.com/docs/ for descriptions.

# For example, if you would like to load Cartpole,
# just replace the above statement with "env = gym.make('CartPole-v1')".

env = stable_baselines3.common.monitor.Monitor(env, log_dir )

callback = EvalCallback(env, log_path=log_dir, deterministic=True)  # For evaluating the performance of the agent periodically and logging the results.
policy_kwargs = dict(activation_fn=torch.nn.ReLU,
                     net_arch=nn_layers)
model = DQN("MlpPolicy", env,policy_kwargs = policy_kwargs,
            learning_rate=learning_rate,
            batch_size=1,  # for simplicity, we are not doing batch update.
            buffer_size=1,  # size of experience of replay buffer. Set to 1 as batch update is not done
            learning_starts=1,  # learning starts immediately!
            gamma=0.99,  # discount facto. range is between 0 and 1.
            tau = 1,  # the soft update coefficient for updating the target network
            target_update_interval=1,  # update the target network immediately.
            train_freq=(1,"step"),  # train the network at every step.
            max_grad_norm = 10,  # the maximum value for the gradient clipping
            exploration_initial_eps = 1,  # initial value of random action probability
            exploration_fraction = 0.5,  # fraction of entire training period over which the exploration rate is reduced
            gradient_steps = 1,  # number of gradient steps
            seed = 1,  # seed for the pseudo random generators
            verbose=0)  # Set verbose to 1 to observe training logs. We encourage you to set the verbose to 1.

# You can also experiment with other RL algorithms like A2C, PPO, DDPG etc.
# Refer to  https://stable-baselines3.readthedocs.io/en/master/guide/examples.html
# for documentation. For example, if you would like to run DDPG, just replace "DQN" above with "DDPG".

/home/iv/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
env_name = 'LunarLander-v2'
env = gym.make(env_name)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [9]:
!pip list | grep gym

gym                           0.26.2
gym-notices                   0.0.8
gymnasium                     0.29.1


In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8+4, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        # return F.log_softmax(x)
        return x

class DQN2():
    def __init__(self):
        self.net = Net()
        learning_rate = 1e-3
        self.optimizer = optim.Adam(self.net.parameters(), lr=learning_rate)
        

    def predict(self, observation, deterministic=False):
        actions = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
        dct = {}
        dct[tuple([1, 0, 0, 0])] = 0
        dct[tuple([0, 1, 0, 0])] = 1
        dct[tuple([0, 0, 1, 0])] = 2
        dct[tuple([0, 0, 0, 1])] = 3

        Q = []
        for act in actions:
            Q.append([self.net.forward(torch.cat((torch.tensor(observation), torch.tensor(act)))), act])
#             actions = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
#             Q.append(torch.cat((torch.tensor(observation), torch.tensor(actions[act]))))
        Q.sort()
#         print(Q[0][1])
        action = dct[tuple(Q[0][1])]

        states = True # не знаю зачем нужно
        return (action, states)

    def learn(self, training_set):
        dct = {}
        dct[0] = [1, 0, 0, 0]
        dct[1] = [0, 1, 0, 0]
        dct[2] = [0, 0, 1, 0]
        dct[3] = [0, 0, 0, 1]
        
        out = []
        Y = []
        for i in training_set:
            temp = []
            temp.extend(i[0])
            temp.extend(dct[i[1]])
            out.append(temp)
            Y.append(i[-1])
    
        x = torch.tensor(out)
        
        self.net.train()
        for i in range(1):
            loss_mean = []
            for xx, YY in zip(x, Y): 
                xx = torch.tensor(xx, dtype=torch.float32)
                YY = torch.tensor(YY, dtype=torch.float32)
                self.optimizer.zero_grad()
                output = self.net(xx)
                mse_loss = torch.nn.MSELoss()
                loss = mse_loss(output.float(), -torch.tensor(YY).float())
                loss_mean.append(loss)
#                 print('loss: ', loss)
                loss.backward()
                self.optimizer.step()
            print(torch.tensor(loss_mean).sum())
                
                
# model2.net.train()
# for i in range(1):
#     loss_mean = []
#     for xx, YY in zip(x, Y): 
#         xx = torch.tensor(xx, dtype=torch.float32)
#         YY = torch.tensor(YY, dtype=torch.float32)
#         model2.optimizer.zero_grad()
#         output = model2.net(xx)
#         mse_loss = torch.nn.MSELoss()
#         loss = mse_loss(output.float(), -torch.tensor(YY).float())
#         loss_mean.append(loss)
# #         print('loss: ', loss)
#         loss.backward()
#         model2.optimizer.step()
#     print(torch.tensor(loss_mean).sum())
        
        
        # training_set = (observation, action, reward)
#         self.net.train()
#         for batch_idx, (observation, action, reward) in enumerate(training_set):
#             if not observation.any():
#                 observation = env.reset()[0]
# #             print(env.reset()[0])
#             self.optimizer.zero_grad()
# #             print('obs', observation)
# #             print('act', action)
#             actions = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
#             output = self.net.forward(torch.cat((torch.tensor(observation), torch.tensor(actions[action]))))
# #             print('out', type(output))
# #             print('out', type(reward))
#             mse_loss = torch.nn.MSELoss()
#             loss = mse_loss(output.float(), torch.tensor(reward).float())
# #             loss = (output - torch.tensor(reward)) ** 2
#             print('loss: ', loss)

#             loss.backward()
#             self.optimizer.step()

model2 = DQN2()
# model2.learn(training_set)
#######
env = gym.make(env_name, render_mode="rgb_array")
vid = VideoRecorder(env, path=f"video/{env_name}_pretraining.mp4")
print(vid.frames_per_sec)
observation = env.reset()[0]

total_reward = 0
done = False

training_set = []

while not done:
    frame = env.render()
    vid.capture_frame()
    action, states = model2.predict(observation, True)
    observation_old = observation
  # print(action)
    observation, reward, done, info, _ = env.step(action)
    total_reward += reward
    training_set.append([observation, action, reward])
vid.close()
env.close()
print(f"\nTotal reward: {total_reward}")

# show video
html = render_mp4(f"video/{env_name}_pretraining.mp4")
HTML(html)

50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -472.29412864884125


In [11]:
for i in range(200):
    model2.learn(training_set)

/tmp/ipykernel_48905/714987317.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xx = torch.tensor(xx, dtype=torch.float32)
/tmp/ipykernel_48905/714987317.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = mse_loss(output.float(), -torch.tensor(YY).float())
/home/iv/anaconda3/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961.3877)
tensor(12961

In [12]:
model2.learn(training_set)

tensor(12961.3877)


/tmp/ipykernel_48905/714987317.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xx = torch.tensor(xx, dtype=torch.float32)
/tmp/ipykernel_48905/714987317.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = mse_loss(output.float(), -torch.tensor(YY).float())


In [12]:
import pickle
with open('model2_400.pkl', 'rb') as f:
    # Загружаем список из файла
    model2 = pickle.load(f)

In [ ]:
for k in range(400//50, 1000000):
    for j in range(50):
        for i in range(5):
            model2.learn(training_set)

        env = gym.make(env_name, render_mode="rgb_array")
        vid = VideoRecorder(env, path=f"video/{env_name}_pretraining.mp4")
        print(vid.frames_per_sec)
        observation = env.reset()[0]

        total_reward = 0
        done = False

        training_set = []

        while not done:
            frame = env.render()
            vid.capture_frame()
            action, states = model2.predict(observation, True)
            observation_old = observation
          # print(action)
            observation, reward, done, info, _ = env.step(action)
            total_reward += reward
            training_set.append([observation, action, reward])
        vid.close()
        env.close()
        print(f"\nTotal reward: {total_reward}")

        # show video
        html = render_mp4(f"video/{env_name}_pretraining.mp4")
        HTML(html)
        
        path = 'model2_' + str(50 * k) + '.pkl'
        with open(path, 'wb') as f:
            pickle.dump(model2, f)

/tmp/ipykernel_55357/714987317.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xx = torch.tensor(xx, dtype=torch.float32)
/tmp/ipykernel_55357/714987317.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = mse_loss(output.float(), -torch.tensor(YY).float())
/home/iv/anaconda3/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(14291.5195)
tensor(3063.9275)
tensor(887.5344)
tensor(307.4045)
tensor(127.0315)
50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -262.1523685068847
tensor(1218.7465)
tensor(767.4717)
tensor(714.4163)
tensor(700.6576)
tensor(691.0622)
50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -52.394611139413044
tensor(5339.0942)
tensor(1657.6926)
tensor(975.8292)
tensor(960.1232)
tensor(956.1204)
50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -167.24663360073265
tensor(10953.9512)
tensor(10706.7871)
tensor(10686.6953)
tensor(10670.7539)
tensor(10658.0078)
50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



...

Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -574.56473617645
tensor(287.3697)
tensor(726.3547)
tensor(2964.0723)
tensor(2158.5024)
tensor(389.5988)
50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -197.95978932558418
tensor(24121.7363)
tensor(12969.7510)
tensor(10214.5049)
tensor(8261.2920)
tensor(6791.2710)
50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -210.74538409184157
tensor(4202.8276)
tensor(1610.1473)
tensor(1122.8230)
tensor(905.5720)
tensor(828.5694)
50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -92.77184992724179
tensor(572.7192)
tensor(730.4559)
tensor(584.7354)
tensor(493.9849)
tensor(440.1833)
50
Moviepy - Building video video/LunarLander-v2_pretraining.mp4.
Moviepy - Writing video video/LunarLander-v2_pretraining.mp4



Moviepy - Done !
Moviepy - video ready video/LunarLander-v2_pretraining.mp4

Total reward: -407.3112582943305
tensor(8356.3447)
tensor(3162.8989)
tensor(566.4643)


In [ ]:
env = gym.make(env_name, render_mode="rgb_array")
vid = VideoRecorder(env, path=f"video/{env_name}_pretraining.mp4")
print(vid.frames_per_sec)
observation = env.reset()[0]

total_reward = 0
done = False

training_set = []

while not done:
    frame = env.render()
    vid.capture_frame()
    action, states = model2.predict(observation, True)
    observation_old = observation
  # print(action)
    observation, reward, done, info, _ = env.step(action)
    total_reward += reward
    training_set.append([observation, action, reward])
vid.close()
env.close()
print(f"\nTotal reward: {total_reward}")

# show video
html = render_mp4(f"video/{env_name}_pretraining.mp4")
HTML(html)

In [28]:
training_set

[[array([-0.00748262,  1.4001019 , -0.3842022 , -0.2532761 ,  0.01043133,
          0.12269433,  0.        ,  0.        ], dtype=float32),
  1,
  -2.222957198835813],
 [array([-0.01135521,  1.3937941 , -0.39581314, -0.28043857,  0.01888825,
          0.169154  ,  0.        ,  0.        ], dtype=float32),
  1,
  -2.73928862330328],
 [array([-0.01532173,  1.3868897 , -0.40758237, -0.30703086,  0.02969735,
          0.21620214,  0.        ,  0.        ], dtype=float32),
  1,
  -2.9437100729961956],
 [array([-0.0193819 ,  1.3793924 , -0.41930708, -0.33352473,  0.04284549,
          0.26298693,  0.        ,  0.        ], dtype=float32),
  1,
  -3.149397683997931],
 [array([-0.02352619,  1.3713013 , -0.42984265, -0.36011913,  0.05809211,
          0.3049607 ,  0.        ,  0.        ], dtype=float32),
  1,
  -3.2502814571856207],
 [array([-0.02776795,  1.362602  , -0.44201502, -0.38743386,  0.07577744,
          0.35373905,  0.        ,  0.        ], dtype=float32),
  1,
  -3.638561667745817

In [23]:
import pickle

In [31]:
with open('Jupyter projects/Lunar_lander_traning_set.pkl', 'wb') as f:
    pickle.dump(training_set, f)

In [403]:
dct = {}
dct[0] = [1, 0, 0, 0]
dct[1] = [0, 1, 0, 0]
dct[2] = [0, 0, 1, 0]
dct[3] = [0, 0, 0, 1]

out = []
Y = []
for i in training_set:
    temp = []
    temp.extend(i[0])
    temp.extend(dct[i[1]])
    out.append(temp)
    Y.append(i[-1])
    
x = torch.tensor(out)

In [404]:
model2.net.train()
for i in range(1):
    loss_mean = []
    for xx, YY in zip(x, Y): 
        xx = torch.tensor(xx, dtype=torch.float32)
        YY = torch.tensor(YY, dtype=torch.float32)
        model2.optimizer.zero_grad()
        output = model2.net(xx)
        mse_loss = torch.nn.MSELoss()
        loss = mse_loss(output.float(), -torch.tensor(YY).float())
        loss_mean.append(loss)
        print('loss: ', loss)
        loss.backward()
        model2.optimizer.step()
    print(torch.tensor(loss_mean).sum())

loss:  tensor(5.0906e-06, grad_fn=<MseLossBackward0>)
loss:  tensor(0.0096, grad_fn=<MseLossBackward0>)
loss:  tensor(0.0081, grad_fn=<MseLossBackward0>)
loss:  tensor(0.0756, grad_fn=<MseLossBackward0>)
loss:  tensor(1.1618, grad_fn=<MseLossBackward0>)
loss:  tensor(2.3713, grad_fn=<MseLossBackward0>)
loss:  tensor(3.0579, grad_fn=<MseLossBackward0>)
loss:  tensor(4.1628, grad_fn=<MseLossBackward0>)
loss:  tensor(4.9944, grad_fn=<MseLossBackward0>)
loss:  tensor(6.3461, grad_fn=<MseLossBackward0>)
loss:  tensor(6.7893, grad_fn=<MseLossBackward0>)
loss:  tensor(9.0527, grad_fn=<MseLossBackward0>)
loss:  tensor(9.8744, grad_fn=<MseLossBackward0>)
loss:  tensor(11.6132, grad_fn=<MseLossBackward0>)
loss:  tensor(13.8205, grad_fn=<MseLossBackward0>)
loss:  tensor(15.5122, grad_fn=<MseLossBackward0>)
loss:  tensor(17.2541, grad_fn=<MseLossBackward0>)
loss:  tensor(18.4819, grad_fn=<MseLossBackward0>)
loss:  tensor(20.2939, grad_fn=<MseLossBackward0>)
loss:  tensor(22.1902, grad_fn=<MseLossB

/tmp/ipykernel_5502/641151555.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xx = torch.tensor(xx, dtype=torch.float32)
/tmp/ipykernel_5502/641151555.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = mse_loss(output.float(), -torch.tensor(YY).float())
